We begin by importing the relevant libraries.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cvxpy as cp

Having done so, we set a couple of parameters. In particular, we work on a version of the interval $[0,L]$ which has been discretized to contain $K$ equally-spaced points. In other words, we consider probability measures supported on  $\{0,\frac{L}{K-1},\dots, \frac{L(K-2)}{K-1}, L\}$

In [ ]:
K = 128 #Number of points
L = 1 #Len
print(range(K//4))
print(range(3*K//4,K))

Now, we define some possible choices for the measures $\mu$ and $\nu$ which we use in the report. To generate the graphics in the report, we chose $\mu$ to be `mu_left` and $\nu$ to be `mu_right`

In [ ]:
mu_rand = np.random.random((K))
mu_rand /= np.sum(mu_rand)                                     #Normalize to get a probability measure
plt.ylim([0, 0.04])                                      
plt.bar([d/K for d in range(K)],mu_rand, width=1/K, align = 'edge') #Uncomment to get plots
plt.draw()                                                     #Uncomment to get plots

In [ ]:
mu_linear = np.zeros((K))
for i in range(K):
    mu_linear[i] += i                                                 #Have the mass assigned to each point be proportional to the distance from the origin
mu_linear /= np.sum(mu_linear)                                        #Normalize to get a probability measure
plt.bar([d/K for d in range(K)],mu_linear, width=1/K, align = 'edge')
plt.ylim([0, 0.04])
plt.draw()

In [ ]:
mu_unif = np.zeros((K))
for i in range(K):                                                  #We can change the range here
    mu_unif[i] += 1
mu_unif /= np.sum(mu_unif)                                          #Normalize to get a probability measure
plt.bar([d/K for d in range(K)],mu_unif, width=1/K, align = 'edge')
plt.ylim([0, 0.04])
plt.draw()

Now we define a $K\times K$ matrix whose entries are the pairwise distances between points. 

In [ ]:
distances = np.zeros((K,K))
for i in range(K):
    for j in range(K):
        distances[i][j] = L*np.abs(i-j)/(K-1)

Having done so, we use the `cvxpy` library to define a function which executes one step of the WROF minimizing movement scheme. This function takes positional arguments `mu`, `nu`, and `tau`, where `mu` is a source measure (i.e. the status quo before doing one step of the minimizing movement scheme), `nu` is the target measure, representing a distribution of real data, and `tau` is the step length. 

In his thesis, Milne indicates that `tau` should be some constant multiple of the $d_1$ distance between `mu` and `nu`, but in the interest of minimizing computation time, I used a constant step size, as solving for the $d_1$ distance requires another linear program. 

Additionally, I've included a keyword argument `normalize_d1` which allows us to replace the $d_2(\rho,\nu)$ term in the minimizing movement scheme with $ 

In [ ]:
def WROF_step(mu,nu,tau, normalize_d1 = False):
    gamma1 = cp.Variable((K, K))                                                        #Start with two cvxpy variables
    gamma2 = cp.Variable((K, K))                                                            #representing transport plans.
    u = np.ones((K,1))                                                                  #Define a vector of ones (useful for computing marginals efficiently).
    U = ([0 <= gamma1, 0 <= gamma2,                                                     #Introduce constraints, starting with forcing entries of gamma to be >=0.
        cp.matmul(gamma1, u) == np.reshape(mu,(K,1)),                                   #Enforce that gamma1 has first marginal mu.
        cp.matmul(gamma1.T, u) == cp.matmul(gamma2, u),                                 #Enforce that the second marginal of gamma1 and the first 
                                                                                            #marginal of gamma2 agree.
        cp.matmul(gamma2.T,u) == np.reshape(nu,(K,1))])                                 #Enforce that the second marginal of gamma2 is nu.              

    if normalize_d1 == False:                                                           #Standard case.
        objective = (cp.Minimize( cp.sum(cp.multiply(gamma1,distances**2))/(2*tau)      #Minimize the appropriate objective functional, subject to the given constraints. 
                                 + cp.sum(cp.multiply(gamma2,distances)) ))
    else:
        objective = (cp.Minimize( cp.sum(cp.multiply(gamma1,distances**2))/(2*tau)      #This case is similar.
                                  + cp.sum(cp.multiply(gamma2,distances**(1+tau))) ))
    prob = cp.Problem(objective, U)                                                     #Now use cvxpy to solve the problem.
    result = prob.solve()
    rho_WROF = np.reshape(np.matmul(gamma1.T.value, u), K)                              #Rewrite the result as a probability vector.
    return rho_WROF                                                                     #Return the probability vector rho. 



Finally, we implement the WROF scheme, and return the result at each step as a graph, which can be either saved or shown. 

In [ ]:
rho = mu_rand                                                           #Start with initial condition mu_rand.
for u in range(35):                                                     #Set a number of steps to iterate over.
    rho = WROF_step(rho,mu_linear, 0.005)                               #Run a WROF step.
    plt.bar([d/K for d in range(K)],rho, width=1/K, align = 'edge')     #Return the result as a bar plot.
    plt.ylim([0, 0.018])                                                #Set a reasonable scale for the y-axis. 
    plt.show()                                                          #OPTIONAL: Show the plot.
    #plt.savefig("num_step"+str(u)+".jpg")                              #OPTIONAL: Uncomment to save each step in a local directory.                             
    plt.clf()                                                           #Clear the plot before running the next step.